# <span style="color:Maroon">ПРОЕКТ: Оптимизация производственных расходов</span>

<div style="background-color:#f3f3f3; border:solid #363636 1.5px; padding: 15px">    
Чтобы оптимизировать производственные расходы, металлургический комбинат решил уменьшить потребление электроэнергии на этапе обработки стали.<br><br>    
<b>Цель данного проекта</b> - Необходимо построить модель, которая предскажет температуру стали<br><br>    
Заказчику важны:<br>
<li>Модель со значением метрики качества <b>MAE</b> не более <b>8.7</b>.<br><br>    
<span style="color:Maroon"><b>1 Загрузить и подготовить данные:</b></span><br><br>
<li>Проверить состав предоставленной выбороки;
<li>Проанализировать данные;
<li>Провести предобработку данных;
<li>Описать результаты.<br><br>
<span style="color:Maroon"><b>2 Обучить модель и выбрать лучшую:</b></span><br><br>
<li>Обучите разные модели на базовых настройках;
<li>Обучите разные модели на с подбором гиперпараметров;
<li>Подобрать для моделей оптимальные гиперпараметры;
<li>Оценить качество моделей кросс-валидацией;
<li>Выбрать лучшую модель
<li>Сделать выводы.<br><br>
<span style="color:Maroon"><b>3 Тестирование лучшей модели:</b></span><br><br>
<li>Проверить модель на тестовой выборке;
<li>Проанализировать предсказания выбранной модели;
<li>Написать выводы и обосновать выбор.
</div>

<div style="background-color:#f3f3f3; border:solid #363636 1.5px; padding: 15px">
создам стиль для отображения табличных данных
</div>

In [3]:
cell_hover     = {'selector': 'td:hover',
                  'props'   : [('background', '#9E4447'), 
                               ('color', '#ffffff')]}           # формат выделенной ячейки

row_hover      = {'selector': 'tr:hover',
                  'props'   : [('background', '#808080'), 
                               ('color', '#ffffff')]}           # формат выделенной строки

color_row_even = {'selector': 'tr:nth-of-type(even)',
                  'props'   : [('background', '#D9D9D9'),
                               ('color', 'black')]}             # формат нечетных строк

color_row_odd  = {'selector': 'tr:nth-of-type(odd)',
                  'props'   : [('background', '#ffffff'),
                               ('color', '#363636')]}           # формат четных строк

index_names    = {'selector': 'th',
                  'props'   : [('background', '#363636'), 
                               ('color', '#ffffff'),  
                               ('text-align','center')]}        # формат заголовка и индекса

border_inner   = {'selector': 'td',
                  'props'   : [('border','1px dashed #363636')]}# формат границы таблицы

border_outer   = {'selector': '',
                  'props'   : [('border','2px solid #363636')]} # формат границы таблицы

caption        = {'selector': 'caption',
                  'props'   : [('color', '#363636'), 
                               ('font-size', '15px')]}

# передаю в переменную для дальнейшего использования
styler = [cell_hover, color_row_even, color_row_odd, index_names, row_hover, border_inner, border_outer, caption]

## Загрузка и подготовка данных

In [4]:
# библиотеки
import re
import pandas as pd 
import numpy as np
import seaborn as sns
import warnings 
from matplotlib import pyplot as plt
from time import time

# обработка
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# модели
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMRegressor

# метрики
from sklearn.metrics import mean_absolute_error

# настройки и параметры
from tqdm import notebook
RANDOM     = 1123581321
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

In [5]:
data_arc       = pd.read_csv('data_arc.csv')
data_bulk      = pd.read_csv('data_bulk.csv')
data_bulk_time = pd.read_csv('data_bulk_time.csv')
data_gas       = pd.read_csv('data_gas.csv')
data_temp      = pd.read_csv('data_temp.csv')
data_wire      = pd.read_csv('data_wire.csv')
data_wire_time = pd.read_csv('data_wire_time.csv')

In [6]:
data_flow      = [data_arc, data_bulk, data_bulk_time, data_gas, data_temp, data_wire, data_wire_time]
data_flow_name = ['data_arc', 'data_bulk', 'data_bulk_time', 'data_gas', 'data_temp', 'data_wire', 'data_wire_time']

<div style="background-color:#f3f3f3; border:solid #363636 1.5px; padding: 15px">
Посмотрим на качество предоставленных данных
</div>

In [8]:
for name, data in enumerate(data_flow):
    print(f'Набор данных {data_flow_name[name]}')
    print()
    display(data.info())
    print('='*100)

Набор данных data_arc

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14876 entries, 0 to 14875
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   key                   14876 non-null  int64  
 1   Начало нагрева дугой  14876 non-null  object 
 2   Конец нагрева дугой   14876 non-null  object 
 3   Активная мощность     14876 non-null  float64
 4   Реактивная мощность   14876 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 581.2+ KB


None

Набор данных data_bulk

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3129 entries, 0 to 3128
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   key      3129 non-null   int64  
 1   Bulk 1   252 non-null    float64
 2   Bulk 2   22 non-null     float64
 3   Bulk 3   1298 non-null   float64
 4   Bulk 4   1014 non-null   float64
 5   Bulk 5   77 non-null     float64
 6   Bulk 6   576 non-null    float64
 7   Bulk 7   25 non-null     float64
 8   Bulk 8   1 non-null      float64
 9   Bulk 9   19 non-null     float64
 10  Bulk 10  176 non-null    float64
 11  Bulk 11  177 non-null    float64
 12  Bulk 12  2450 non-null   float64
 13  Bulk 13  18 non-null     float64
 14  Bulk 14  2806 non-null   float64
 15  Bulk 15  2248 non-null   float64
dtypes: float64(15), int64(1)
memory usage: 391.2 KB


None

Набор данных data_bulk_time

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3129 entries, 0 to 3128
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   key      3129 non-null   int64 
 1   Bulk 1   252 non-null    object
 2   Bulk 2   22 non-null     object
 3   Bulk 3   1298 non-null   object
 4   Bulk 4   1014 non-null   object
 5   Bulk 5   77 non-null     object
 6   Bulk 6   576 non-null    object
 7   Bulk 7   25 non-null     object
 8   Bulk 8   1 non-null      object
 9   Bulk 9   19 non-null     object
 10  Bulk 10  176 non-null    object
 11  Bulk 11  177 non-null    object
 12  Bulk 12  2450 non-null   object
 13  Bulk 13  18 non-null     object
 14  Bulk 14  2806 non-null   object
 15  Bulk 15  2248 non-null   object
dtypes: int64(1), object(15)
memory usage: 391.2+ KB


...

None

Набор данных data_wire

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3081 entries, 0 to 3080
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   key     3081 non-null   int64  
 1   Wire 1  3055 non-null   float64
 2   Wire 2  1079 non-null   float64
 3   Wire 3  63 non-null     float64
 4   Wire 4  14 non-null     float64
 5   Wire 5  1 non-null      float64
 6   Wire 6  73 non-null     float64
 7   Wire 7  11 non-null     float64
 8   Wire 8  19 non-null     float64
 9   Wire 9  29 non-null     float64
dtypes: float64(9), int64(1)
memory usage: 240.8 KB


None

Набор данных data_wire_time

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3081 entries, 0 to 3080
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   key     3081 non-null   int64 
 1   Wire 1  3055 non-null   object
 2   Wire 2  1079 non-null   object
 3   Wire 3  63 non-null     object
 4   Wire 4  14 non-null     object
 5   Wire 5  1 non-null      object
 6   Wire 6  73 non-null     object
 7   Wire 7  11 non-null     object
 8   Wire 8  19 non-null     object
 9   Wire 9  29 non-null     object
dtypes: int64(1), object(9)
memory usage: 240.8+ KB


None

<div style="background-color:#f3f3f3; border:solid #363636 1.5px; padding: 15px">
После первого осмотра данных выявлены следующие замечания:<br>
<li>основной датасет <b>data_temp</b>, в нем находится информация по температуре на момент завершения технологического процесса плавки металла, последняя запись о замере температуры в партии является целевым признаком. Соответственно построение финального датасета будет ориентированно именно на этот набор данных;
<li>В датасете <b>data_temp</b> есть большое количество пропусков относительно общего набора данных, восстанавливать эти данные нет смысла и может только навредить моделированию, т.к. целевой ориентир может быть искажен;
<li>Общие замечание по наборам данных: наличие большого количества пропусков в данных, название столбцов не в "питонском формате", тип данных <b><i>object</i></b> где информация о времени, необходимо изменить тип.
<li>Далее пройдемся по каждому набору данных более детально, но предварительно сразу изменю название столбцов и поменяю тип данных.
</div>

In [9]:
data_arc.columns  = ['key', 'time_start_heat', 'time_end_heat', 'active_pwr', 'reactive_pwr']
data_gas.columns  = ['key', 'gas']
data_temp.columns = ['key', 'time_measure', 'temperature']

In [10]:
# функция для изменения формата даты
def object_to_date(data):
    for column in data.columns:
        if data[column].dtype == 'object':
            data[column] = pd.to_datetime(data[column], format="%Y-%m-%d %H:%M:%S")
    return data

In [11]:
# функция для переименования столбцов в "питонский формат"
def columns_rename(data):
    columns_new = []
    for column in data.columns:
        columns_new.append(column.replace(' ', '_').lower())
    data.columns = columns_new
    return data

In [12]:
for data in data_flow:
    columns_rename(data)
    object_to_date(data) 

In [13]:
for name, data in enumerate(data_flow):
    display(data.head(10).style\
                         .set_caption(f'Набор данных {data_flow_name[name]}')\
                         .set_table_styles(styler))
    display(data.info())
    print('='*100)

,key,time_start_heat,time_end_heat,active_pwr,reactive_pwr
0,1,2019-05-03 11:02:14,2019-05-03 11:06:02,0.976059,0.687084
1,1,2019-05-03 11:07:28,2019-05-03 11:10:33,0.805607,0.520285
2,1,2019-05-03 11:11:44,2019-05-03 11:14:36,0.744363,0.498805
3,1,2019-05-03 11:18:14,2019-05-03 11:24:19,1.659363,1.062669
4,1,2019-05-03 11:26:09,2019-05-03 11:28:37,0.692755,0.414397
5,2,2019-05-03 11:34:14,2019-05-03 11:36:31,0.438063,0.283043
6,2,2019-05-03 11:38:50,2019-05-03 11:44:28,1.296415,0.892914
7,2,2019-05-03 11:46:19,2019-05-03 11:48:25,0.490377,0.305281
8,2,2019-05-03 11:49:48,2019-05-03 11:53:18,0.827743,0.516874
9,3,2019-05-03 12:06:54,2019-05-03 12:11:34,1.062053,0.671494


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14876 entries, 0 to 14875
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   key              14876 non-null  int64         
 1   time_start_heat  14876 non-null  datetime64[ns]
 2   time_end_heat    14876 non-null  datetime64[ns]
 3   active_pwr       14876 non-null  float64       
 4   reactive_pwr     14876 non-null  float64       
dtypes: datetime64[ns](2), float64(2), int64(1)
memory usage: 581.2 KB


None

,key,bulk_1,bulk_2,bulk_3,bulk_4,bulk_5,bulk_6,bulk_7,bulk_8,bulk_9,bulk_10,bulk_11,bulk_12,bulk_13,bulk_14,bulk_15
0,1,nan,nan,nan,43.000000,nan,nan,nan,nan,nan,nan,nan,206.000000,nan,150.000000,154.000000
1,2,nan,nan,nan,73.000000,nan,nan,nan,nan,nan,nan,nan,206.000000,nan,149.000000,154.000000
2,3,nan,nan,nan,34.000000,nan,nan,nan,nan,nan,nan,nan,205.000000,nan,152.000000,153.000000
3,4,nan,nan,nan,81.000000,nan,nan,nan,nan,nan,nan,nan,207.000000,nan,153.000000,154.000000
4,5,nan,nan,nan,78.000000,nan,nan,nan,nan,nan,nan,nan,203.000000,nan,151.000000,152.000000
5,6,nan,nan,nan,117.000000,nan,nan,nan,nan,nan,nan,nan,204.000000,nan,201.000000,154.000000
6,7,nan,nan,nan,117.000000,nan,nan,nan,nan,nan,nan,nan,204.000000,nan,152.000000,154.000000
7,8,nan,nan,nan,99.000000,nan,nan,nan,nan,nan,nan,nan,410.000000,nan,252.000000,153.000000
8,9,nan,nan,nan,117.000000,nan,nan,nan,nan,nan,nan,nan,107.000000,nan,99.000000,203.000000
9,10,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,203.000000,nan,102.000000,204.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3129 entries, 0 to 3128
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   key      3129 non-null   int64  
 1   bulk_1   252 non-null    float64
 2   bulk_2   22 non-null     float64
 3   bulk_3   1298 non-null   float64
 4   bulk_4   1014 non-null   float64
 5   bulk_5   77 non-null     float64
 6   bulk_6   576 non-null    float64
 7   bulk_7   25 non-null     float64
 8   bulk_8   1 non-null      float64
 9   bulk_9   19 non-null     float64
 10  bulk_10  176 non-null    float64
 11  bulk_11  177 non-null    float64
 12  bulk_12  2450 non-null   float64
 13  bulk_13  18 non-null     float64
 14  bulk_14  2806 non-null   float64
 15  bulk_15  2248 non-null   float64
dtypes: float64(15), int64(1)
memory usage: 391.2 KB


...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3081 entries, 0 to 3080
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   key     3081 non-null   int64  
 1   wire_1  3055 non-null   float64
 2   wire_2  1079 non-null   float64
 3   wire_3  63 non-null     float64
 4   wire_4  14 non-null     float64
 5   wire_5  1 non-null      float64
 6   wire_6  73 non-null     float64
 7   wire_7  11 non-null     float64
 8   wire_8  19 non-null     float64
 9   wire_9  29 non-null     float64
dtypes: float64(9), int64(1)
memory usage: 240.8 KB


None

,key,wire_1,wire_2,wire_3,wire_4,wire_5,wire_6,wire_7,wire_8,wire_9
0,1,2019-05-03 11:11:41,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,2,2019-05-03 11:46:10,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2,3,2019-05-03 12:13:47,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
3,4,2019-05-03 12:48:05,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
4,5,2019-05-03 13:18:15,2019-05-03 13:32:06,NaT,NaT,NaT,NaT,NaT,NaT,NaT
5,6,2019-05-03 13:59:02,2019-05-03 13:59:54,NaT,NaT,NaT,NaT,NaT,NaT,NaT
6,7,2019-05-03 14:28:46,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
7,8,2019-05-03 15:03:42,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
8,9,2019-05-03 15:42:43,2019-05-03 15:58:12,NaT,NaT,NaT,NaT,NaT,NaT,NaT
9,10,2019-05-03 16:20:25,2019-05-03 16:21:02,NaT,NaT,NaT,NaT,NaT,NaT,NaT


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3081 entries, 0 to 3080
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   key     3081 non-null   int64         
 1   wire_1  3055 non-null   datetime64[ns]
 2   wire_2  1079 non-null   datetime64[ns]
 3   wire_3  63 non-null     datetime64[ns]
 4   wire_4  14 non-null     datetime64[ns]
 5   wire_5  1 non-null      datetime64[ns]
 6   wire_6  73 non-null     datetime64[ns]
 7   wire_7  11 non-null     datetime64[ns]
 8   wire_8  19 non-null     datetime64[ns]
 9   wire_9  29 non-null     datetime64[ns]
dtypes: datetime64[ns](9), int64(1)
memory usage: 240.8 KB


None

<div style="background-color:#f3f3f3; border:solid #363636 1.5px; padding: 15px">
Первые изменения внесены, теперь необходимо познакомиться с составом информации по партиям в каждом наборе данных, чтобы понять насколько большие различия в них, т.к. некоторые таблицы содержат уже агрегированные данные для каждой партии, а некоторые несут дополнительную информацию по итерациям нагрева, необходимо понять сколько всего уникальных партий
</div>

In [14]:
for name, data in enumerate(data_flow):
    print(f'Количество партий {len(data["key"].unique())} в наборе данных {data_flow_name[name]}')

Количество партий 3214 в наборе данных data_arc
Количество партий 3129 в наборе данных data_bulk
Количество партий 3129 в наборе данных data_bulk_time
Количество партий 3239 в наборе данных data_gas
Количество партий 3216 в наборе данных data_temp
Количество партий 3081 в наборе данных data_wire
Количество партий 3081 в наборе данных data_wire_time


<div style="background-color:#f3f3f3; border:solid #363636 1.5px; padding: 15px">
Как видно из информации выше, в каждом датасете имеется разное количество партий, за исключение только информации о материалах, но это очевидно т.к. они сильно зависимы. Это очень странная ситуация, которая говорит о том, что выгрузка была произведена скорее всего не верно, качество предоставленных данных оставляет желать лучшего. Здесь бы я, наверное, обратился к тому, кто выгружал их, но за неимением такой возможности продолжим работать с тем что есть.
</div>

## Анализ данных

### Анализ данных по замерам температуры

<div style="background-color:#f3f3f3; border:solid #363636 1.5px; padding: 15px">
Первый датасет для анализа выбран не случайно, как я говорил выше это основной набор информации, на котором будет строится аналитика. В этом датасете мне необходимо получить информацию для каждой партии о первом замере температуры и последнем, для этого промаркирую набор данных
</div>

In [16]:
target = data_temp.copy()